In [2]:
import numpy
import matplotlib.pyplot as plt
import pandas
import math
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Activation,Dropout
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import Conv1D, GlobalAveragePooling1D, MaxPooling1D

from tensorflow.keras.backend import argmax
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

from tensorflow.keras.utils import to_categorical

In [3]:
# convert an array of values into a dataset matrix
def create_dataset(dataset, look_back=1):
    dataX, dataY = [], []
    for i in range(len(dataset)-look_back-1):
        a = dataset[i:(i+look_back), 0]
        dataY.append(dataset[i + look_back, 0]) 
        dataX.append(a)
    return numpy.array(dataX), numpy.array(dataY)

In [4]:
import datasets 
from datasets import load_ypred, load_usps, load_mrec

Y=load_ypred()

Y.shape
Y.iloc[10,0]

MM (4030, 1)
YPRED: (4030, 1)


2

In [5]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler(feature_range=(0, 0.999))
YM = scaler.fit_transform(Y)


In [6]:
look_back = 10
trainX, trainY = create_dataset(YM, look_back)


In [7]:
trainY

array([0.1332, 0.7992, 0.3996, ..., 0.1332, 0.0666, 0.5994])

In [8]:
trainX

array([[0.0666, 0.5994, 0.1332, ..., 0.1332, 0.7992, 0.5994],
       [0.5994, 0.1332, 0.7992, ..., 0.7992, 0.5994, 0.1332],
       [0.1332, 0.7992, 0.3996, ..., 0.5994, 0.1332, 0.7992],
       ...,
       [0.0666, 0.5994, 0.1332, ..., 0.7326, 0.0666, 0.5994],
       [0.5994, 0.1332, 0.0666, ..., 0.0666, 0.5994, 0.1332],
       [0.1332, 0.0666, 0.5994, ..., 0.5994, 0.1332, 0.0666]])

In [9]:
# reshape input to be [samples, time steps, features]
trainX = numpy.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1]))
trainX.shape
#testX = numpy.reshape(testX, (testX.shape[0], 1, testX.shape[1]))

(4019, 1, 10)

In [10]:
#le = preprocessing.LabelEncoder()
#YN=utils.column_or_1d(Ydataset, warn=True)
#YO=le.fit_transform(trainY)
#YO


In [25]:
# create and fit the LSTM network
model = Sequential()
model.add(LSTM(4, input_shape=(1, look_back)))
model.add(Dense(1))
model.compile(loss='mean_absolute_error'
              , optimizer='adam')
model.fit(trainX, trainY, epochs=10, verbose=2)

Epoch 1/10
4019/4019 - 0s - loss: 0.2563
Epoch 2/10
4019/4019 - 0s - loss: 0.1939
Epoch 3/10
4019/4019 - 0s - loss: 0.1370
Epoch 4/10
4019/4019 - 0s - loss: 0.0971
Epoch 5/10
4019/4019 - 0s - loss: 0.0706
Epoch 6/10
4019/4019 - 0s - loss: 0.0589
Epoch 7/10
4019/4019 - 0s - loss: 0.0550
Epoch 8/10
4019/4019 - 0s - loss: 0.0531
Epoch 9/10
4019/4019 - 0s - loss: 0.0522
Epoch 10/10
4019/4019 - 0s - loss: 0.0515


In [26]:
trainX[1:8,:]

array([[[0.5994, 0.1332, 0.7992, 0.3996, 0.8658, 0.5994, 0.1332, 0.7992,
         0.5994, 0.1332]],

       [[0.1332, 0.7992, 0.3996, 0.8658, 0.5994, 0.1332, 0.7992, 0.5994,
         0.1332, 0.7992]],

       [[0.7992, 0.3996, 0.8658, 0.5994, 0.1332, 0.7992, 0.5994, 0.1332,
         0.7992, 0.3996]],

       [[0.3996, 0.8658, 0.5994, 0.1332, 0.7992, 0.5994, 0.1332, 0.7992,
         0.3996, 0.8658]],

       [[0.8658, 0.5994, 0.1332, 0.7992, 0.5994, 0.1332, 0.7992, 0.3996,
         0.8658, 0.5994]],

       [[0.5994, 0.1332, 0.7992, 0.5994, 0.1332, 0.7992, 0.3996, 0.8658,
         0.5994, 0.1332]],

       [[0.1332, 0.7992, 0.5994, 0.1332, 0.7992, 0.3996, 0.8658, 0.5994,
         0.1332, 0.7992]]])

In [27]:
nn=model.predict(trainX[1:10,:])
print('ll')
print(nn)

ll
[[0.81976426]
 [0.40435225]
 [0.8882693 ]
 [0.58376896]
 [0.14225036]
 [0.80790144]
 [0.61237615]
 [0.1057815 ]
 [0.8197643 ]]


In [1]:
score = model.evaluate(trainX, trainY, verbose=1)

print('Test loss:', score)
print('Test accuracy:', score)


NameError: name 'model' is not defined